In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing

In [ ]:
from quantpoian.pipeline.filters import Q1500US

In [ ]:
universe = Q1500US()

In [ ]:
from quantopian.pipeline.data import morningstar

In [ ]:
sector = morningstar.asset_classification.morningstar_sector_code.latest

In [ ]:
energy_sector = sector.eq(309)

In [ ]:
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarValue

In [ ]:
dollar_volume = AverageDollarVolume(window_length=30)
high_dollar_volume = dollar_volume.percentile_between(90,100)
top_open_price = USEquityPricing.open.latest.top(50,mask=high_dollar_volume) # top 50 securities with high open price among with top 10% dollar volume
high_close_price = USEquityPricing.close.latest.percentile_between(90,100,mask=top_open_price) # top 50 securities with high close price among top 10% 

In [2]:
def male_pipeline():
    base_universe = Q1500US()
    energy_sector = sector.eq(309)
    base_energy = base_universe & energy_sector
    dollar_volume = AverageDollarVolume(window_length=30)
    high_dollar_volume = dollar_volume.percentile_between(95,100)
    top_half_base_energy = base_energy & high_dollar_volume
    mean_10 = SimpleMovingAverage(inputs= [USEquityPricing.close], window_length=10,mask=top_half_base_energy)
    mean_30 = SimpleMovingAverage(inputs= [USEquityPricing.close], window_length=30,mask=top_half_base_energy)
    percent_difference = (mean_10 - mean_30)/mean_30
    shorts = percent_difference < 0
    longs = percent_difference > 0
    securirties_to_trade = (longs | shorts)
    return Pipeline(columns = {
            'longs': longs,
            'shorts': shorts,
            'percent_diff':percent_difference
    },screen=securities_to_trade)

In [ ]:
result = run_pipeline(make_pipeline(),'2015-05-05','2015-05-05')
result

In [ ]:
from quantopian.algorithm import attach_pipeline,pipeline_output
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import AverageDollarVolume,SimpleMovingAverage
from quantopian.pipeline.filters.morningstar import Q1500US
from quantopian.pipeline.data import morningstar


# init->  before_trading_start (uses compute weights) -> my_rebalance
def initialize(context):
    schedule_function(my_rebalance,date_rules.week_start(),time_rules.market_open(hours=1))
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe,'my_pipeline')



def my_rebalance(context,data):
    for security in context.portfolio.positions:
        if security not in context.longs and security not in context.shorts and data.can_trade(security):
            order_target_percent(security,0)
    for security in context.longs:
        if data.can_trade(security):
            order_target_percent(security,context.long_weight)

    for security in context.shorts:
        if data.can_trade(security):
            order_target_percent(security,context.short_weight)

def my_compute_weights(context):
    
    if len(context.longs)==0:
        long_weight = 0
    else:
        long_weight = 0.5 / len(context.longs)
  
    if len(context.shorts)==0:
        short_weight = 0
    else:
        short_weight = 0.5 / len(context.shorts)
    
    return (long_weight,short_weight)
    
def before_trading_start(context,data):
    context.output = pipeline_output('my_pipeline')

    context.longs = context.output[context.output['longs']].index.tolist()
    context.shorts = context.output[context.output['shorts']].index.tolist()
    context.long_weight,context.short_weight = my_compute_weights(context)

def make_pipeline():
    
    # Universe Q1500US
    base_universe = Q1500US()
    
    # Energy Sector
    sector = morningstar.asset_classification.morningstar_sector_code.latest
    energy_sector = sector.eq(309)
    
    # Make Mask of 1500US and Energy
    base_energy = base_universe & energy_sector
    
    # Dollar Volume (30 Days) Grab the Info
    dollar_volume = AverageDollarVolume(window_length=30)
    
    # Grab the top 5% in avg dollar volume
    high_dollar_volume = dollar_volume.percentile_between(95,100)
     
    # Combine the filters
    top_five_base_energy = base_energy & high_dollar_volume
    
    # 10 day mean close
    mean_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10,mask=top_five_base_energy)
    
    # 30 day mean close
    mean_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30,mask=top_five_base_energy)
    
    # Percent Difference
    percent_difference = (mean_10-mean_30)/mean_30
    
    # List of Shorts
    shorts = percent_difference < 0
    
    # List of Longs
    longs = percent_difference > 0
    
    # Final Mask/Filter for anything in shorts or longs
    securities_to_trade = (shorts | longs)
    
    # Return Pipeline
    return Pipeline(columns={
        'longs':longs,
        'shorts':shorts,
        'perc_diff':percent_difference
    },screen=securities_to_trade)
    